In [ ]:
# default_exp data.tsloader

# TimeSeriesLoader
> Data Loader for Time Series data

In [ ]:
#hide
from nbdev import *
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import warnings
from collections.abc import Mapping
from typing import Dict, List, Optional, Union

import numpy as np
import torch as t
from fastcore.foundation import patch
from torch.utils.data import DataLoader

from nixtlats.data.tsdataset import TimeSeriesDataset, WindowsDataset

## Inherited `DataLoader` from `pytorch` 

In [ ]:
#export
class TimeSeriesLoader(DataLoader):

    def __init__(self, dataset: Union[TimeSeriesDataset, WindowsDataset], 
                 eq_batch_size: bool = False, 
                 **kwargs) -> 'TimeSeriesLoader':
        """Wraps the pytorch `DataLoader` with a special collate function 
        for the `TimeSeriesDataset` ouputs.
        
        The TimeSeriesDataset constructs all the trainable windows 
        of `batch_size` series. The number of windows can be greater 
        or smaller than the `batch_size`. For this reason, 
        an additional boolean parameter, `eq_batch_size` is included 
        that if `True` samples `batch_size` windows randomly, 
        while `False` returns all windows.
        
        Parameters
        ----------
        dataset: TimeSeriesDataset
            Stored time series.
        eq_batch_size: bool
            If `True` samples `batch_size` windows randomly,
            while `False` or `batch_size=None` returns all windows.
        """
        if 'collate_fn' in kwargs.keys():
            warnings.warn(
                'This class wraps the pytorch `DataLoader` with a '
                'special collate function. If you want to use yours '
                'simply use `DataLoader`. Removing collate_fn'
            )
            kwargs.pop('collate_fn')
            
        kwargs_ = {**kwargs, **dict(collate_fn=self._collate_fn)}
        DataLoader.__init__(self, dataset=dataset, **kwargs_)
        self.eq_batch_size = eq_batch_size
        self.w_idxs: Optional[np.ndarray] = None

In [ ]:
#export
@patch
def _check_batch_size(self: TimeSeriesLoader, batch: t.Tensor):
    complete_batch = batch
    if self.w_idxs is not None:
        complete_batch = batch[self.w_idxs]
    
    return complete_batch

In [ ]:
#export
@patch
def _collate_fn(self: TimeSeriesLoader, batch: Union[List, Dict[str, t.Tensor], t.Tensor]):
    """Special collate fn for the `TimeSeriesDataset`.
    
    Notes
    -----
    [1] Adapted from https://github.com/pytorch/pytorch/blob/master/torch/utils/data/_utils/collate.py.
    """
    elem = batch[0]
    if len(batch) == 1:
        return {key: self._check_batch_size(elem[key]) for key in elem}
    
    elem_type = type(elem)
    
    if isinstance(elem, t.Tensor):
        out = None
        if t.utils.data.get_worker_info() is not None:
            # If we're in a background process, concatenate directly into a
            # shared memory tensor to avoid an extra copy
            numel = sum([x.numel() for x in batch])
            storage = elem.storage()._new_shared(numel)
            out = elem.new(storage)
        complete_batch = t.cat(batch, out=out)
        return self._check_batch_size(complete_batch)
    
    elif isinstance(elem, Mapping):
        n_windows = elem['Y'].size(0)
        if self.eq_batch_size and self.batch_size is not None:
            self.w_idxs = np.random.choice(n_windows, size=self.batch_size, 
                                           replace=(n_windows < self.batch_size))
        return {key: self.collate_fn([d[key] for d in batch]) for key in elem}

    raise TypeError(f'Unknown {elem_type}')

### Tests WindowsDataset and TimeSeriesDataset

In [ ]:
from nixtlats.data.utils import create_synthetic_tsdata

In [ ]:
def test_eq_batch_size(dataset, batch_size, loader_class):
    # Check returns batch_size tensors
    loader = loader_class(dataset=dataset, batch_size=batch_size, eq_batch_size=True)
    sizes = [batch['Y'].size(0) == batch_size for batch in loader]
    
    assert all(sizes), 'Unexpected batch sizes.'

In [ ]:
def test_eq_batch_size_order(dataset, batch_size, loader_class):
    #This test only works for TimeSeriesDataset class
    loader = loader_class(dataset=dataset, batch_size=batch_size, eq_batch_size=True)
    for batch in loader:
        idxs = batch['idxs']
        dataset_batch = dataset[idxs.numpy().tolist()]
        for key in batch.keys():
            assert t.equal(batch[key], dataset_batch[key]), (
                f'Batch and dataset batch differ, key {key}'
            )

#### Complete timeseries dataset

In [ ]:
Y_df, X_df, S_df = create_synthetic_tsdata(sort=True)
dataset = TimeSeriesDataset(S_df=S_df, Y_df=Y_df, X_df=X_df,
                            input_size=5,
                            output_size=2)
dataloader = TimeSeriesLoader(dataset=dataset, batch_size=12, 
                              eq_batch_size=False, shuffle=True)

for batch in dataloader:
    batch

In [ ]:
test_eq_batch_size(dataset, 32, TimeSeriesLoader)

In [ ]:
test_eq_batch_size_order(dataset, 32, TimeSeriesLoader)

#### Windowed timeseries dataset

In [ ]:
Y_df, X_df, S_df = create_synthetic_tsdata(sort=True)
dataset = WindowsDataset(S_df=S_df, Y_df=Y_df, X_df=X_df,
                         input_size=5,
                         output_size=2,
                         sample_freq=1)
dataloader = TimeSeriesLoader(dataset=dataset, batch_size=12, 
                              eq_batch_size=False, shuffle=True)

for batch in dataloader:
    batch

In [ ]:
test_eq_batch_size(dataset, 32, TimeSeriesLoader)

## Faster implemention

In [ ]:
#export
class FastTimeSeriesLoader:
    """
    A DataLoader-like object for a set of tensors that can be much faster than
    TensorDataset + DataLoader because dataloader grabs individual indices of
    the dataset and calls cat (slow).
    Source: https://discuss.pytorch.org/t/dataloader-much-slower-than-manual-batching/27014/6
    
    Notes
    -----
    [1] Adapted from https://github.com/hcarlens/pytorch-tabular/blob/master/fast_tensor_data_loader.py.
    """
    def __init__(self, dataset: TimeSeriesDataset, batch_size: int = 32, 
                 eq_batch_size: bool = False,
                 shuffle: bool = False) -> 'FastTimeSeriesLoader':
        """Initialize a FastTimeSeriesLoader.
        
        The TimeSeriesDataset constructs all the trainable windows 
        of `batch_size` series. The number of windows can be greater 
        or smaller than the `batch_size`. For this reason, 
        an additional boolean parameter, `eq_batch_size` is included 
        that if `True` samples `batch_size` windows randomly, 
        while `False` returns all windows.
        
        Parameters
        -----------
        dataset: TimeSeriesDataset
            Stored time series.
        batch_size: int
            Batch size to load.
        shuffle: bool 
            If `True`, shuffle the data *in-place* whenever an
            iterator is created out of this object.
        """
        self.dataset = dataset
        self.dataset_len = len(dataset)
        self.batch_size = batch_size
        self.eq_batch_size = batch_size
        self.shuffle = shuffle
        self.idxs = np.arange(self.dataset_len)

        # Calculate # batches
        n_batches, remainder = divmod(self.dataset_len, self.batch_size)
        if remainder > 0:
            n_batches += 1
        self.n_batches = n_batches
        self.w_idxs: Optional[np.ndarray] = None

In [ ]:
#export
@patch
def __iter__(self: FastTimeSeriesLoader):
    if self.shuffle:
        self.idxs = np.random.permutation(self.dataset_len)

    self.i = 0
    return self

In [ ]:
#export
@patch
def _check_batch_size(self: FastTimeSeriesLoader, batch: t.Tensor):
    complete_batch = batch
    if self.w_idxs is not None:
        complete_batch = batch[self.w_idxs]
    return complete_batch

In [ ]:
#export
@patch
def __next__(self: FastTimeSeriesLoader):
    if self.i >= self.dataset_len:
        raise StopIteration
    idxs = self.idxs[self.i:(self.i + self.batch_size)].tolist()
    batch = self.dataset[idxs]
    self.i += self.batch_size
    
    n_windows = batch['Y'].size(0)
    if self.eq_batch_size and self.batch_size is not None:
        self.w_idxs = np.random.choice(n_windows, size=self.batch_size, 
                                     replace=(n_windows < self.batch_size))
    
    return {key: self._check_batch_size(batch[key]) for key in batch}

In [ ]:
#export
@patch
def __len__(self: FastTimeSeriesLoader):
    return self.n_batches

### Tests WindowsDataset and TimeSeriesDataset

#### Complete timeseries dataset

In [ ]:
Y_df, X_df, S_df = create_synthetic_tsdata(sort=True)
dataset = TimeSeriesDataset(S_df=S_df, Y_df=Y_df, X_df=X_df,
                            input_size=5,
                            output_size=2)
dataloader = FastTimeSeriesLoader(dataset=dataset, batch_size=12, 
                                  eq_batch_size=False, shuffle=True)

for batch in dataloader:
    batch

In [ ]:
test_eq_batch_size(dataset, 32, FastTimeSeriesLoader)

In [ ]:
test_eq_batch_size_order(dataset, 32, FastTimeSeriesLoader)

#### Windowed timeseries dataset

In [ ]:
Y_df, X_df, S_df = create_synthetic_tsdata(sort=True)
dataset = WindowsDataset(S_df=S_df, Y_df=Y_df, X_df=X_df,
                         input_size=5,
                         output_size=2,
                         sample_freq=1)
dataloader = FastTimeSeriesLoader(dataset=dataset, batch_size=12, 
                                  eq_batch_size=False, shuffle=True)

In [ ]:
for batch in dataloader:
    batch

In [ ]:
test_eq_batch_size(dataset, 32, FastTimeSeriesLoader)

### Performance comparison

In [ ]:
dataloader = TimeSeriesLoader(dataset=dataset, batch_size=12, shuffle=True)
fast_dataloader = FastTimeSeriesLoader(dataset=dataset, batch_size=12, shuffle=True)

In [ ]:
%timeit -n 50 -r 3  [batch for batch in dataloader]

In [ ]:
%timeit -n 50 -r 3 [batch for batch in fast_dataloader]